# Mosaico 1996

Script trabajado por Francisco Albornoz (2021-04-13) / Última modificación / Mosaicos Patagonia

2021-08-05 13:07:09 

In [ ]:
import ee
import geemap
import geemap.chart as chart
import os
import folium

<img src=https://www.ecopaysen.cl/wp-content/uploads/2019/12/ecopaysen_marca_web.png>

In [ ]:
Map = geemap.Map(center = [-45, -70], zoom = 4)
Map2 = geemap.Map(center = [-45, -70], zoom = 4)
Map3 = geemap.Map(center = [-45, -70], zoom = 4)
Map4 = geemap.Map(center = [-45, -70], zoom = 4)
Map5 = geemap.Map(center = [-45, -70], zoom = 4)
Map6 = geemap.Map(center = [-45, -70], zoom = 4)
Map7 = geemap.Map(center = [-45, -70], zoom = 4)
Map8 = geemap.Map(center = [-45, -70], zoom = 4)
Map9 = geemap.Map(center = [-45, -70], zoom = 4)
Map10 = geemap.Map(center = [-45, -70], zoom = 4)

## Área de Estudio

In [ ]:
work_dir = os.path.expanduser('../../../Data/shapes/')
patagonia = os.path.join(work_dir, 'patagonia_nueva.shp')
patagonia_grande = geemap.shp_to_ee(patagonia)

## Colors Bars

In [ ]:
import geemap.colormaps as cm

In [ ]:
# cm.palettes.terrain

In [ ]:
color_ndvi = cm.palettes.ndvi

In [ ]:
color_ndwi = cm.palettes.ndwi

In [ ]:
color_terrain = cm.palettes.terrain

In [ ]:
color_glaciar = cm.palettes.PuBuGn_r

In [ ]:
color_nubes = cm.palettes.inferno

In [ ]:
# cm.list_colormaps()

In [ ]:
# cm.plot_colormaps(width=12, height=0.4)

## Funciones

In [ ]:
def add_ee_layer(self, ee_image_object, vis_params, name):
  map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
  folium.raster_layers.TileLayer(
      tiles=map_id_dict['tile_fetcher'].url_format,
      attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
      name=name,
      overlay=True,
      control=True
  ).add_to(self)

folium.Map.add_ee_layer = add_ee_layer

In [ ]:
def addNDVI(image): 
    ndvi = image.normalizedDifference(['B4', 'B3']).rename('B11')
    return image.addBands(ndvi)

In [ ]:
def addNDWI(image):
    ndwi = image.normalizedDifference(['B2','B4']).rename('NDWI')
    return image.addBands(ndwi)

In [ ]:
def addNDGI(image):
    ndgi = image.normalizedDifference(['B2','B3']).rename('NDGI')
    return image.addBands(ndgi)

In [ ]:
def addSAVI(image):
    savi = image.expression(
        '((b4 - b3)/(b4 + b3 + 0.5))*1.5', {
            'b3':image.select('B3'),
            'b4':image.select('B4')
        }) 
    savi= savi.rename('SAVI')
    return image.addBands(savi)

In [ ]:
def addNDSI(image):
    ndsi = image.normalizedDifference(['B2','B5']).rename('NDSI')
    return image.addBands(ndsi)

In [ ]:
def addDate(image):
    img_date = ee.Date(image.date())
    img_date = ee.Number.parse(img_date.format('YYYYMM'))
    return image.addBands(ee.Image(img_date).rename('date').toInt())

In [ ]:
def Cloud30(image):
    scored = ee.Algorithms.Landsat.simpleCloudScore(image)
    return image.updateMask(scored.select(['cloud']).lt(30)) 

In [ ]:
def Cloud20(image):
    scored = ee.Algorithms.Landsat.simpleCloudScore(image)
    return image.updateMask(scored.select(['cloud']).lt(20)) 

In [ ]:
def Cloud10(image):
    scored = ee.Algorithms.Landsat.simpleCloudScore(image)
    return image.updateMask(scored.select(['cloud']).lt(10))

In [ ]:
def addCloud(image):
    nube = ee.Algorithms.Landsat.simpleCloudScore(image)
    return image.addBands(ee.Image(nube.select(['cloud'])).rename('nube'))

## Colecciones

### 1996

In [ ]:
start_date = '1995-10-01'
end_date = '1996-03-31'

l5_1996 = ee.ImageCollection('LANDSAT/LT05/C01/T1_TOA') \
    .filterBounds(patagonia_grande) \
    .filterDate(start_date, end_date)

In [ ]:
vizParams = {
  'bands': ['B4', 'B3', 'B2'],
  'min': 0,
  'max': 0.5,
  'gamma': [0.95, 1.1, 1]
}

In [ ]:
vis_nieve = {
  'bands': ['B5', 'B3', 'B2'],
  'min': 0,
  'max': 0.5,
  'gamma': [0.95, 1.1, 1]
}

In [ ]:
coleccion1996 = l5_1996.map(addNDVI).map(addNDWI).map(addNDGI).map(addSAVI).map(addNDSI).map(addDate).map(addCloud)

In [ ]:
QM_NDVI_1996 = coleccion1996.qualityMosaic('B11') # Normalized difference vegetation Index
QM_NDWI_1996 = coleccion1996.qualityMosaic('NDWI') # Normalize difference water index
QM_SAVI_1996 = coleccion1996.qualityMosaic('SAVI') # Soil Adjusted Vegetation Index 
QM_NDGI_1996 = coleccion1996.qualityMosaic('NDGI') # Normalized difference glacier index
QM_NDSI_1996 = coleccion1996.qualityMosaic('NDSI') # Normalized difference snow index

In [ ]:
Map.addLayer(QM_NDVI_1996, vizParams, 'QM de NDVI 1996')
Map.addLayer(QM_NDWI_1996, vizParams, 'QM de NDWI 1996')
Map.addLayer(QM_SAVI_1996, vizParams, 'QM de SAVI 1996')
Map.addLayer(QM_NDGI_1996, vis_nieve, 'QM de NDGI 1996')
Map.addLayer(QM_NDSI_1996, vis_nieve, 'QM de NDSI 1996')
Map 

In [ ]:
c1996_sn = coleccion1996.map(Cloud20)
NDVI_1996_sn = c1996_sn.qualityMosaic('B11')
NDWI_1996_sn = c1996_sn.qualityMosaic('NDWI')
SAVI_1996_sn = c1996_sn.qualityMosaic('SAVI')
NDGI_1996_sn = c1996_sn.qualityMosaic('NDGI')
NDSI_1996_sn = c1996_sn.qualityMosaic('NDSI')

In [ ]:
Map2.addLayer(NDVI_1996_sn, vizParams, 'QMsn de NDVI 1996')
Map2.addLayer(NDWI_1996_sn, vizParams, 'QMsn de NDWI 1996')
Map2.addLayer(SAVI_1996_sn, vizParams, 'QMsn de SAVI 1996')
Map2.addLayer(NDGI_1996_sn, vis_nieve, 'QMsn de NDGI 1996')
Map2.addLayer(NDSI_1996_sn, vis_nieve, 'QMsn de NDSI 1996')
Map2

### 1997

In [ ]:
start_date = '1996-10-01'
end_date = '1997-03-31'

l5_1997 = ee.ImageCollection('LANDSAT/LT05/C01/T1_TOA') \
    .filterBounds(patagonia_grande) \
    .filterDate(start_date, end_date)

In [ ]:
coleccion1997 = l5_1997.map(addNDVI).map(addNDWI).map(addNDGI).map(addSAVI).map(addNDSI).map(addDate).map(addCloud)

In [ ]:
QM_NDVI_1997 = coleccion1997.qualityMosaic('B11') # Normalized difference vegetation Index
QM_NDWI_1997 = coleccion1997.qualityMosaic('NDWI') # Normalize difference water index
QM_SAVI_1997 = coleccion1997.qualityMosaic('SAVI') # Soil Adjusted Vegetation Index 
QM_NDGI_1997 = coleccion1997.qualityMosaic('NDGI') # Normalized difference glacier index
QM_NDSI_1997 = coleccion1997.qualityMosaic('NDSI') # Normalized difference snow index

In [ ]:
Map.addLayer(QM_NDVI_1997, vizParams, 'QM de NDVI 1997')
Map.addLayer(QM_NDWI_1997, vizParams, 'QM de NDWI 1997')
Map.addLayer(QM_SAVI_1997, vizParams, 'QM de SAVI 1997')
Map.addLayer(QM_NDGI_1997, vis_nieve, 'QM de NDGI 1997')
Map.addLayer(QM_NDSI_1997, vis_nieve, 'QM de NDSI 1997')
Map 

In [ ]:
c1997_sn = coleccion1997.map(Cloud20)
NDVI_1997_sn = c1997_sn.qualityMosaic('B11')
NDWI_1997_sn = c1997_sn.qualityMosaic('NDWI')
SAVI_1997_sn = c1997_sn.qualityMosaic('SAVI')
NDGI_1997_sn = c1997_sn.qualityMosaic('NDGI')
NDSI_1997_sn = c1997_sn.qualityMosaic('NDSI')

In [ ]:
Map2.addLayer(NDVI_1997_sn, vizParams, 'QMsn de NDVI 1997')
Map2.addLayer(NDWI_1997_sn, vizParams, 'QMsn de NDWI 1997')
Map2.addLayer(SAVI_1997_sn, vizParams, 'QMsn de SAVI 1997')
Map2.addLayer(NDGI_1997_sn, vis_nieve, 'QMsn de NDGI 1997')
Map2.addLayer(NDSI_1997_sn, vis_nieve, 'QMsn de NDSI 1997')
Map2

### 1998

In [ ]:
start_date = '1997-10-01'
end_date = '1998-03-31'

l5_1998 = ee.ImageCollection('LANDSAT/LT05/C01/T1_TOA') \
    .filterBounds(patagonia_grande) \
    .filterDate(start_date, end_date)

In [ ]:
coleccion1998 = l5_1998.map(addNDVI).map(addNDWI).map(addNDGI).map(addSAVI).map(addNDSI).map(addDate).map(addCloud)

In [ ]:
QM_NDVI_1998 = coleccion1998.qualityMosaic('B11') # Normalized difference vegetation Index
QM_NDWI_1998 = coleccion1998.qualityMosaic('NDWI') # Normalize difference water index
QM_SAVI_1998 = coleccion1998.qualityMosaic('SAVI') # Soil Adjusted Vegetation Index 
QM_NDGI_1998 = coleccion1998.qualityMosaic('NDGI') # Normalized difference glacier index
QM_NDSI_1998 = coleccion1998.qualityMosaic('NDSI') # Normalized difference snow index

In [ ]:
Map3.addLayer(QM_NDVI_1998, vizParams, 'QM de NDVI 1998')
Map3.addLayer(QM_NDWI_1998, vizParams, 'QM de NDWI 1998')
Map3.addLayer(QM_SAVI_1998, vizParams, 'QM de SAVI 1998')
Map3.addLayer(QM_NDGI_1998, vis_nieve, 'QM de NDGI 1998')
Map3.addLayer(QM_NDSI_1998, vis_nieve, 'QM de NDSI 1998')
# Map3

In [ ]:
c1998_sn = coleccion1998.map(Cloud20)
NDVI_1998_sn = c1998_sn.qualityMosaic('B11')
NDWI_1998_sn = c1998_sn.qualityMosaic('NDWI')
SAVI_1998_sn = c1998_sn.qualityMosaic('SAVI')
NDGI_1998_sn = c1998_sn.qualityMosaic('NDGI')
NDSI_1998_sn = c1998_sn.qualityMosaic('NDSI')

In [ ]:
Map4.addLayer(NDVI_1998_sn, vizParams, 'QMsn de NDVI 1998')
Map4.addLayer(NDWI_1998_sn, vizParams, 'QMsn de NDWI 1998')
Map4.addLayer(SAVI_1998_sn, vizParams, 'QMsn de SAVI 1998')
Map4.addLayer(NDGI_1998_sn, vis_nieve, 'QMsn de NDGI 1998')
Map4.addLayer(NDSI_1998_sn, vis_nieve, 'QMsn de NDSI 1998')
Map4

### 1999 

In [ ]:
start_date = '1998-10-01'
end_date = '1999-03-31'

l5_1999 = ee.ImageCollection('LANDSAT/LT05/C01/T1_TOA') \
    .filterBounds(patagonia_grande) \
    .filterDate(start_date, end_date)

In [ ]:
coleccion1999 = l5_1999.map(addNDVI).map(addNDWI).map(addNDGI).map(addSAVI).map(addNDSI).map(addDate).map(addCloud)

In [ ]:
QM_NDVI_1999 = coleccion1999.qualityMosaic('B11') # Normalized difference vegetation Index
QM_NDWI_1999 = coleccion1999.qualityMosaic('NDWI') # Normalize difference water index
QM_SAVI_1999 = coleccion1999.qualityMosaic('SAVI') # Soil Adjusted Vegetation Index 
QM_NDGI_1999 = coleccion1999.qualityMosaic('NDGI') # Normalized difference glacier index
QM_NDSI_1999 = coleccion1999.qualityMosaic('NDSI') # Normalized difference snow index

In [ ]:
Map3.addLayer(QM_NDVI_1999, vizParams, 'QM de NDVI 1999')
Map3.addLayer(QM_NDWI_1999, vizParams, 'QM de NDWI 1999')
Map3.addLayer(QM_SAVI_1999, vizParams, 'QM de SAVI 1999')
Map3.addLayer(QM_NDGI_1999, vis_nieve, 'QM de NDGI 1999')
Map3.addLayer(QM_NDSI_1999, vis_nieve, 'QM de NDSI 1999')
# Map3

In [ ]:
c1999_sn = coleccion1999.map(Cloud20)
NDVI_1999_sn = c1999_sn.qualityMosaic('B11')
NDWI_1999_sn = c1999_sn.qualityMosaic('NDWI')
SAVI_1999_sn = c1999_sn.qualityMosaic('SAVI')
NDGI_1999_sn = c1999_sn.qualityMosaic('NDGI')
NDSI_1999_sn = c1999_sn.qualityMosaic('NDSI')

In [ ]:
Map4.addLayer(NDVI_1999_sn, vizParams, 'QMsn de NDVI 1999')
Map4.addLayer(NDWI_1999_sn, vizParams, 'QMsn de NDWI 1999')
Map4.addLayer(SAVI_1999_sn, vizParams, 'QMsn de SAVI 1999')
Map4.addLayer(NDGI_1999_sn, vis_nieve, 'QMsn de NDGI 1999')
Map4.addLayer(NDSI_1999_sn, vis_nieve, 'QMsn de NDSI 1999')
Map4

## Indices por colección

### Índices 1996

In [ ]:
NDVI_1996 = NDVI_1996_sn.select('B11')
NDWI_1996 = NDWI_1996_sn.select('NDWI') 
SAVI_1996 = SAVI_1996_sn.select('SAVI')
NDGI_1996 = NDGI_1996_sn.select('NDGI')
NDSI_1996 = NDSI_1996_sn.select('NDSI')

In [ ]:
Map7.addLayer(NDVI_1996, {'palette': color_terrain}, 'QMsn de NDVI 1996')
Map7.addLayer(NDWI_1996, {'palette': color_ndwi}, 'QMsn de NDWI 1996')
Map7.addLayer(SAVI_1996, {'palette': color_ndvi}, 'QMsn de SAVI 1996')
# Map7.addLayer(NDGI_1996, {'palette': color_glaciar}, 'QMsn de NDGI 1996')
# Map7.addLayer(NDSI_1996, {'palette': color_ndwi}, 'QMsn de NDSI 1996')
Map7

### Índices 1997

In [ ]:
NDVI_1997 = NDVI_1997_sn.select('B11')
NDWI_1997 = NDWI_1997_sn.select('NDWI') 
SAVI_1997 = SAVI_1997_sn.select('SAVI')
NDGI_1997 = NDGI_1997_sn.select('NDGI')
NDSI_1997 = NDSI_1997_sn.select('NDSI')

In [ ]:
Map7.addLayer(NDVI_1997, {'palette': color_terrain}, 'QMsn de NDVI 1997')
Map7.addLayer(NDWI_1997, {'palette': color_ndwi}, 'QMsn de NDWI 1997')
Map7.addLayer(SAVI_1997, {'palette': color_ndvi}, 'QMsn de SAVI 1997')
# Map7.addLayer(NDGI_1997, {'palette': color_glaciar}, 'QMsn de NDGI 1997')
# Map7.addLayer(NDSI_1997, {'palette': color_ndwi}, 'QMsn de NDSI 1997')
Map7

### Índices 1998

In [ ]:
NDVI_1998 = NDVI_1998_sn.select('B11')
NDWI_1998 = NDWI_1998_sn.select('NDWI') 
SAVI_1998 = SAVI_1998_sn.select('SAVI')
NDGI_1998 = NDGI_1998_sn.select('NDGI')
NDSI_1998 = NDSI_1998_sn.select('NDSI')

In [ ]:
Map8.addLayer(NDVI_1998, {'palette': color_terrain}, 'QMsn de NDVI 1998')
Map8.addLayer(NDWI_1998, {'palette': color_ndwi}, 'QMsn de NDWI 1998')
Map8.addLayer(SAVI_1998, {'palette': color_ndvi}, 'QMsn de SAVI 1998')
# Map8.addLayer(NDGI_1998, {'palette': color_glaciar}, 'QMsn de NDGI 1998')
# Map8.addLayer(NDSI_1998, {'palette': color_ndwi}, 'QMsn de NDSI 1998')
Map8

### Índices 1999

In [ ]:
NDVI_1999 = NDVI_1999_sn.select('B11')
NDWI_1999 = NDWI_1999_sn.select('NDWI') 
SAVI_1999 = SAVI_1999_sn.select('SAVI')
NDGI_1999 = NDGI_1999_sn.select('NDGI')
NDSI_1999 = NDSI_1999_sn.select('NDSI')

In [ ]:
Map8.addLayer(NDVI_1999, {'palette': color_terrain}, 'QMsn de NDVI 1999')
Map8.addLayer(NDWI_1999, {'palette': color_ndwi}, 'QMsn de NDWI 1999')
Map8.addLayer(SAVI_1999, {'palette': color_ndvi}, 'QMsn de SAVI 1999')
# Map8.addLayer(NDGI_1999, {'palette': color_glaciar}, 'QMsn de NDGI 1999')
# Map8.addLayer(NDSI_1999, {'palette': color_ndwi}, 'QMsn de NDSI 1999')
Map8

## Generación de Mosaico

In [ ]:
NDVI_1999_landsat = QM_NDVI_1999.set('SENSOR_ID', 'OLI_TIRS')

In [ ]:
cloud_1999 = ee.Algorithms.Landsat.simpleCloudScore(NDVI_1999_landsat)

In [ ]:
cloud_1999 = cloud_1999.select('cloud')

In [ ]:
palette_nubes = ['#1a9850', '#66bd63', '#a6d96a', '#d9ef8b', '#fee08b', '#fdae61', '#f46d43', '#d73027']

In [ ]:
resultados= [cloud_1999, QM_NDVI_1999]
vis_params2=  [
    {'palette':palette_nubes}, 
    vizParams]
labels = [
    'Nubes 1996',
    'QM NDVI 1996']

In [ ]:
geemap.linked_maps(rows=1, cols=2, height="400px", center=[-45, -75], zoom=4, 
                   ee_objects=resultados, vis_params=vis_params2, labels=labels, label_position="topright")

In [ ]:
def llenado(image):
    return image.where(cloud_1999.gt(20),NDVI_1996_sn)

In [ ]:
img1 = llenado(QM_NDVI_1999)

In [ ]:
img1_landsat = img1.set('SENSOR_ID', 'OLI_TIRS')
cloud_img1 = ee.Algorithms.Landsat.simpleCloudScore(img1_landsat)
cloud_img1 = cloud_img1.select('cloud')

In [ ]:
resultados= [cloud_img1, img1]
vis_params2=  [
    {'palette':palette_nubes}, 
    vizParams]
labels = [
    'Nubes Mosaico 1',
    'Mosaico 1']

In [ ]:
geemap.linked_maps(rows=1, cols=2, height="400px", center=[-45, -75], zoom=4, 
                   ee_objects=resultados, vis_params=vis_params2, labels=labels, label_position="topright")

In [ ]:
def llenado2(image):
    return image.where(cloud_img1.gt(20), NDVI_1997_sn)

In [ ]:
img2 = llenado2(img1)

In [ ]:
img2_landsat = img2.set('SENSOR_ID', 'OLI_TIRS')
cloud_img2 = ee.Algorithms.Landsat.simpleCloudScore(img2_landsat)
cloud_img2 = cloud_img2.select('cloud')

In [ ]:
resultados= [cloud_img2, img2]
vis_params2=  [
    {'palette':palette_nubes}, 
    vizParams]
labels = [
    'Nubes Mosaico 2',
    'Mosaico 2']

In [ ]:
geemap.linked_maps(rows=1, cols=2, height="400px", center=[-45, -75], zoom=4, 
                   ee_objects=resultados, vis_params=vis_params2, labels=labels, label_position="topright")

In [ ]:
def llenado3(image):
    return image.where(cloud_img2.gt(20), NDVI_1998_sn)

In [ ]:
img3 = llenado3(img2)

In [ ]:
img3_landsat = img3.set('SENSOR_ID', 'OLI_TIRS')
cloud_img3 = ee.Algorithms.Landsat.simpleCloudScore(img3_landsat)
cloud_img3 = cloud_img3.select('cloud')

In [ ]:
resultados= [cloud_img3, img3]
vis_params2=  [
    {'palette':palette_nubes}, 
    vizParams]
labels = [
    'Nubes Mosaico 3',
    'Mosaico 3']

In [ ]:
geemap.linked_maps(rows=1, cols=2, height="400px", center=[-45, -75], zoom=4, 
                   ee_objects=resultados, vis_params=vis_params2, labels=labels, label_position="topright")

In [ ]:
def llenado4(image):
    return image.where(cloud_img3.gt(20), NDVI_2018_sn)

In [ ]:
img4 = llenado4(img3)

In [ ]:
img4_landsat = img4.set('SENSOR_ID', 'OLI_TIRS')
cloud_img4 = ee.Algorithms.Landsat.simpleCloudScore(img4_landsat)
cloud_img4 = cloud_img4.select('cloud')

In [ ]:
resultados= [cloud_img4, img4]
vis_params2=  [
    {'palette':palette_nubes}, 
    vizParams]
labels = [
    'Nubes Mosaico 4',
    'Mosaico 4']

In [ ]:
geemap.linked_maps(rows=1, cols=2, height="400px", center=[-45, -75], zoom=4, 
                   ee_objects=resultados, vis_params=vis_params2, labels=labels, label_position="topright")

In [ ]:
def llenado5(image):
    return image.where(cloud_img4.gt(20), NDVI_2017_sn)

In [ ]:
img5 = llenado5(img4)

In [ ]:
img5_landsat = img5.set('SENSOR_ID', 'OLI_TIRS')
cloud_img5 = ee.Algorithms.Landsat.simpleCloudScore(img5_landsat)
cloud_img5 = cloud_img5.select('cloud')

In [ ]:
resultados= [cloud_img5, img5]
vis_params2=  [
    {'palette':palette_nubes}, 
    vizParams]
labels = [
    'Nubes Mosaico 5',
    'Mosaico 5']

In [ ]:
geemap.linked_maps(rows=1, cols=2, height="400px", center=[-45, -75], zoom=4, 
                   ee_objects=resultados, vis_params=vis_params2, labels=labels, label_position="topright")